<a href="https://colab.research.google.com/github/Carlos1729/DGL/blob/main/Node_Classification_with_DGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html

Looking in links: https://data.dgl.ai/wheels/cu116/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.3 MB/s eta 0:00:00


In [2]:
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=27aadd3940ce52586ef079572cdcecd63de386cd1578de7aa3cb04114bffe259
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc

In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


One of the most popular and widely adopted tasks on graph data is node classification, where a model needs to predict the ground truth category of each node. Before graph neural networks, many proposed methods are using either connectivity alone (such as DeepWalk or node2vec), or simple combinations of connectivity and the node’s own features. GNNs, by contrast, offers an opportunity to obtain node representations by combining the connectivity and features of a local neighborhood.

Kipf et al., is an example that formulates the node classification problem as a semi-supervised node classification task. With the help of only a small portion of labeled nodes, a graph neural network (GNN) can accurately predict the node category of the others.

This tutorial will show how to build such a GNN for semi-supervised node classification with only a small number of labels on the Cora dataset, a citation network with papers as nodes and citations as edges. The task is to predict the category of a given paper. Each paper node contains a word count vector as its features, normalized so that they sum up to one, as described in Section 5.2 of the paper.

A DGL Dataset object may contain one or multiple graphs. The Cora dataset used in this tutorial only consists of one single graph.

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [5]:
g = dataset[0]

In [6]:
g1 = dataset[1]

AssertionError: ignored

A DGL graph can store node features and edge features in two dictionary-like attributes called ndata and edata. In the DGL Cora dataset, the graph contains the following node features:

train_mask: A boolean tensor indicating whether the node is in the training set.

val_mask: A boolean tensor indicating whether the node is in the validation set.

test_mask: A boolean tensor indicating whether the node is in the test set.

label: The ground truth node category.

feat: The node features

In [7]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


Defining a Graph Convolutional Network (GCN)

This tutorial will build a two-layer Graph Convolutional Network (GCN). Each layer computes new node representations by aggregating neighbor information.
To build a multi-layer GCN you can simply stack dgl.nn.GraphConv modules, which inherit torch.nn.Module.

In [8]:
g.ndata['feat']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [9]:
g.ndata['feat'].size()

torch.Size([2708, 1433])

In [10]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

In [11]:
g.ndata['feat'].shape[1]

1433

In [12]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [13]:
labels  = g.ndata['label']
labels

tensor([3, 4, 4,  ..., 3, 3, 3])

In [14]:
labels.size()

torch.Size([2708])

In [15]:
train_mask = g.ndata['train_mask']
train_mask
train_mask.size()


torch.Size([2708])

In [16]:
num_true_values = train_mask.sum().item()
num_true_values

140

In [17]:
ls = labels[train_mask]
ls

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6,
        3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,
        3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2,
        2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0,
        6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5,
        5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1])

In [18]:
ls.size()

torch.Size([140])

In [19]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.154 (best 0.154), test acc: 0.169 (best 0.169)
In epoch 5, loss: 1.901, val acc: 0.486 (best 0.558), test acc: 0.510 (best 0.558)
In epoch 10, loss: 1.832, val acc: 0.592 (best 0.592), test acc: 0.591 (best 0.591)
In epoch 15, loss: 1.736, val acc: 0.668 (best 0.668), test acc: 0.672 (best 0.672)
In epoch 20, loss: 1.616, val acc: 0.682 (best 0.684), test acc: 0.696 (best 0.683)
In epoch 25, loss: 1.474, val acc: 0.702 (best 0.702), test acc: 0.722 (best 0.722)
In epoch 30, loss: 1.313, val acc: 0.726 (best 0.726), test acc: 0.747 (best 0.747)
In epoch 35, loss: 1.141, val acc: 0.730 (best 0.734), test acc: 0.744 (best 0.746)
In epoch 40, loss: 0.967, val acc: 0.734 (best 0.736), test acc: 0.745 (best 0.746)
In epoch 45, loss: 0.802, val acc: 0.742 (best 0.742), test acc: 0.749 (best 0.749)
In epoch 50, loss: 0.655, val acc: 0.754 (best 0.754), test acc: 0.758 (best 0.758)
In epoch 55, loss: 0.529, val acc: 0.760 (best 0.760), test acc: 0.761 (best 0

In [20]:
https://www.youtube.com/watch?v=IMcj0GU5md4

SyntaxError: ignored

In [ ]:
https://docs.dgl.ai/en/0.8.x/tutorials/large/L1_large_node_classification.html

In [ ]:
https://docs.dgl.ai/en/0.8.x/tutorials/blitz/1_introduction.html

In [21]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.214 (best 0.214), test acc: 0.232 (best 0.232)
In epoch 5, loss: 1.903, val acc: 0.534 (best 0.534), test acc: 0.605 (best 0.605)
In epoch 10, loss: 1.827, val acc: 0.596 (best 0.596), test acc: 0.640 (best 0.640)
In epoch 15, loss: 1.728, val acc: 0.584 (best 0.604), test acc: 0.647 (best 0.637)
In epoch 20, loss: 1.604, val acc: 0.604 (best 0.604), test acc: 0.656 (best 0.637)
In epoch 25, loss: 1.459, val acc: 0.660 (best 0.660), test acc: 0.707 (best 0.707)
In epoch 30, loss: 1.298, val acc: 0.690 (best 0.690), test acc: 0.721 (best 0.721)
In epoch 35, loss: 1.128, val acc: 0.702 (best 0.702), test acc: 0.732 (best 0.732)
In epoch 40, loss: 0.960, val acc: 0.710 (best 0.710), test acc: 0.747 (best 0.746)
In epoch 45, loss: 0.801, val acc: 0.738 (best 0.738), test acc: 0.759 (best 0.758)
In epoch 50, loss: 0.657, val acc: 0.742 (best 0.742), test acc: 0.766 (best 0.764)
In epoch 55, loss: 0.533, val acc: 0.754 (best 0.754), test acc: 0.773 (best 0

###Node Classification


https://docs.dgl.al/en/0.9.x/guide/training-node.html

The nodes and edges of a DGLGraph can have several user-defined named features for storing graph-specific properties of the nodes and edges. These features cane accessed via the ndata and edata interface

Double-click (or enter) to edit

A feature is created via tensor assignment, which assigns a feature to each node/edge in the graph. The leading dimension of that tensor must be equal to the number of nodes/edges in the graph.

In [22]:
# The feature tensor is in row-major layout each row-slice
# stores the feature of one node or edge

To classify nodes, a graph neural network performs message passing to utilize the node's own features and its neighboring node and edge features.
For message passing, update_all is a high-level API that merges message generation, message aggregation, and node update in a single call, leaving room for optimization as a whole.
The parameters for update_all are a message function, a reduce function, and an update function
The parameters for update all are a message function, a reduce lnction and an update function.



In [23]:
def update_all_example(graph):
    # Store the result in graph.ndata['ft']
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'), fn.sum('n', 'ft'))

    # Call update function outside of update_all
    final_ft = graph.ndata['ft']

    return final_ft

This call will generate the messages m by multiply are node features ft and edge features a, surn up the messages m to update node features ft, and finally multiply ft by 2 to get the result final ft. After the call, DGL will clean the Intermediate messages m.

DGL provides a few built-in graph convolution modules that can perform one round of message passing. In this guide, we choose dgl.nn.pytorch.SAGEConv (also available in MXNet and Tensorflow), the graph convolution module for GraphSAGE.